### Binary Exponentiation

- Imagine we are asked to compute $a^n$

- Of course, in the usual brute force approach, we need to run a loop n times

In [5]:
def binary_exponentiation_naive(base: int, exponent: int) -> int:
    '''
    Time complexity: O(N), where N is the value of the exponent
    '''
    retval = 1
    for i in range(exponent):
        retval *= base
    return retval

binary_exponentiation_naive(2, 13)

8192

- Notice that most of the time comes from the loop! So we want to rewrite the exponentiation such that the loop is minimised. 
    - Imagine we want to do exponentiation of $2^{13}$. Rules:
        - Whenever exponent is odd, multiply the `result` by the base, and subtract one from the exponent 
        - Whenever exponent is even, square the base, and halve the power 
        - Keep going until power reaches 0 

    - Example: Recall that 2^13 previously took 13 steps (because it loops 13 times). Now, we simply take 7 steps!  
        - result = 1; base = 2; power = 13
        - result = 2; base = 2; power = 12
        - result = 2; base = 4; power = 6
        - result = 2; base = 16; power = 3
        - result = 32; base = 16; power = 2
        - result = 32; base = 256; power = 1
        - result = 8192; base = 256; power = 0

    - Time complexity:
        - Let the exponent be $N$
        - We reduce the exponent by half every step, so we have complexity of $O(\log_2(N))$

In [4]:
def binary_exponentiation(base: int, exponent: int) -> int:
    '''
    Time complexity: O(log(N)) due to halving of exponent every step
    '''
    result = 1

    while exponent > 0:
        if exponent % 2 == 1:
            result *= base
        
        exponent = exponent // 2
        base *= base
    
    return result

binary_exponentiation(2, 13)

8192

### Extension: Modular Exponentiation

- A simple extension of this exponentiation is when we wish to take the mod of the exponentiation; $A^B \mod C$
    - $A^B \mod C = (A \mod C)^B \mod C$
        - Write $A$ as some multiple of $C$ plus a remainder $r$; $A = x \cdot C + r$
        - Then $A^2 =  x^2 \cdot C^2 + 2 \cdot r \cdot x \cdot C + r^2$
        - Taking modulo of C, notice that the first 2 terms of $A^2$ are guaranteed to be divisible by $C$
        - Then $A^2 \mod C = r^2 \mod C = (A \mod C)^2 \mod C$
        - The same argument applies to all powers $k$ of A, because if you expand A according to the binomial theorem, all terms will contain C except for the $r^k$ term

In [60]:
def modular_exponentiation(base: int, exponent: int, modulo: int) -> int:
    result = 1

    while exponent > 0:
        if exponent % 2 != 0:
            result = (result * base) % modulo
            exponent -= 1
        
        base = ((base % modulo) * (base % modulo)) % modulo
        exponent = exponent//2
    return result % modulo

In [61]:
import numpy as np

for _ in range(10_000):
    base = int(np.random.randint(1, 50, 1)[0])
    exponent = int(np.random.randint(10, 60, 1)[0])
    modulo = int(np.random.randint(7, 77, 1)[0])

    manual = (base**exponent) % modulo
    function = modular_exponentiation(base, exponent, modulo)
    
    assert manual == function, f'{base=}, {exponent=}, {modulo=}, {manual=}, {function=}'


### Example: Prime Interval

- Given left and right bounds, print all primes within the bounds

In [123]:
def is_prime(num: int) -> bool:
    if num in [0,1]:
        return False
    
    if num in [2,3]:
        return True

    for i in range(2, int(num**0.5 // 1)+1):
        if num % i == 0:
            return False
    return True
 
def primes_in_interval_naive(left: int, right: int) -> list[int]:
    '''
    Time complexity: O(N * sqrt(N)), where N is the interval length between left and right, and the sqrt(N) comes from the primality test for each number
    '''
    results = []
    for i in range(left, right+1):
        # print(f'testing value: {i: ^20}')
        if is_prime(i):
            results.append(i)

    return results

def primes_in_interval_eratosthenes(left: int, right: int) -> np.array:
    '''
    Time complexity: O(N log log(N)). 
    Space complexity: O(N)
    '''
    is_prime = np.ones(right+1)
    is_prime[0], is_prime[1] = 0, 0

    for i in range(2, right+1):
        if is_prime[i]:
            is_prime[i**2 : right+1 : i] = 0
    
    return np.where(is_prime[left:])[0] + left


primes_in_interval_eratosthenes(100, 1000)

array([101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163,
       167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233,
       239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311,
       313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389,
       397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463,
       467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563,
       569, 571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641,
       643, 647, 653, 659, 661, 673, 677, 683, 691, 701, 709, 719, 727,
       733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821,
       823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907,
       911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997])

### Example: Prime Prime

- We define a new type of number called Prime Prime
- A number X is a prime prime if the number of prime numbers from 1 to X (inclusive) is prime 
- How many prime prime numbers are there in some range L and R?

- Naive approach:
    - Iterate through all numbers between 1 and $X$ [$O(X)$]
    - For each value $x \in [1,X]$, check `is_prime(x)` [$O(\sqrt{x})$]. If yes, append to an array
    - Finally, count length of array, and check if length is prime [$O(\sqrt{L})$]
    - $O(X^2 + L)$

- Better approach:
    - Using Eratosthenes sieve; 
        a. init 2 arrays, `is_prime` and `is_prime_prime`
        b. init a counter, `count_cumulative_prime`
        c. Get all primes between 1 and X [$O(X log log X)$]
            - For each value $i$ between 1 and X, we check if `is_prime[i]` (this is just the standard Eratosthenes sieve loop)
            - If `is_prime[i]`
                - increment `count_cumulative_prime`
                - If `is_prime[count_cumulative_prime]`, then set `is_prime_prime[i]` to 1
                - Do the standard sieve while loop, setting all multiples of the prime value to false

In [33]:
import numpy as np

def is_prime(num: int) -> bool:
    if num <= 1:
        return False

    if num in [2,3]:
        return True

    for j in range(2, int((num**0.5) // 1)+1):
        if num % j == 0:
            return False
    return True


def get_prime_prime_naive(left: int, right: int) -> list[int]:
    primes = []
    prime_primes = []
    for num in range(2, right+1):
        if is_prime(num):
            primes.append(num)
            if len(primes) in primes:
                prime_primes.append(num)

    # return primes, prime_primes
    return len([x for x in prime_primes if x >= left])
    
def get_prime_prime_eratosthenes(left: int, right: int) -> list[int]:
    is_prime = np.ones(right+1)
    is_prime[0], is_prime[1] = 0,0 
    
    is_prime_prime = np.zeros(right+1)
    count_primes = 0

    for num in range(2, right+1):
        if is_prime[num]:
            count_primes += 1
            if is_prime[count_primes]:
                is_prime_prime[num] = 1
            
            is_prime[num**2: right+1: num] = 0

    primes = np.where(is_prime)[0]
    prime_primes = primes[is_prime_prime[primes].astype('bool')]
    return prime_primes

# get_prime_prime_naive(1, 100)
# get_prime_prime_eratosthenes(1, 100)

array([ 3,  5, 11, 17, 31, 41, 59, 67, 83])